In [23]:
import pandas as pd

In [24]:
#pd.set_option('display.max_columns', None)

clean_file = pd.read_csv('output/listing_details_cleaned.csv')
clean_file

,Tỉnh/Thành phố,Thành phố/Quận/Huyện/Thị xã,Xã/Phường/Thị trấn,Đường phố,Chi tiết,Nguồn thông tin,Tình trạng giao dịch,Thời điểm giao dịch/rao bán,Thông tin liên hệ,Giá rao bán/giao dịch,...,Kích thước chiều dài (m),Số mặt tiền tiếp giáp,Hình dạng,Độ rộng ngõ/ngách nhỏ nhất (m),Khoảng cách tới trục đường chính (m),Mục đích sử dụng đất,Yếu tố khác,Tọa độ (vĩ độ),Tọa độ (kinh độ),Hình ảnh của bài đăng
0,Khánh Hòa,Nha Trang,Phường Phương Sơn,Đường Hải Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,2.800000e+09,...,9.0,1,Nở hậu,3.0,NaN,Đất ở,"vị trí nhà tiếp giáp hẻm rộng 3m, cách đường 2...",12.252321,109.178597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
1,Hà Nội,Nam Từ Liêm,Phường Tây Mỗ,Đường Tây Mỗ,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,5.700000e+09,...,NaN,1,Chữ nhật,2.2,NaN,Đất ở,"nhà dân xây, phân lô, cần bán gấp giá rẻ | nhà...",21.004792,105.749786,"[""https://file4.batdongsan.com.vn/crop/600x315..."
2,Hồ Chí Minh,Phú Nhuận,Phường 5,Đường Thích Quảng Đức,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,07/07/2025,NaN,1.050000e+10,...,19.0,1,Chữ nhật,5.0,NaN,Đất ở,"nhà 1 trệt 1 lững 2 lầu sân thượng - 9pn, 9wc,...",10.805619,106.682205,"[""https://file4.batdongsan.com.vn/crop/600x315..."
3,Hà Nội,Cầu Giấy,Phường Quan Hoa,Đường Nguyễn Khánh Toàn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,1.820000e+10,...,NaN,1,Chữ nhật,3.5,NaN,Đất ở,"bán nhà phố nguyễn khánh toàn, cầu giấy, khu p...",21.037317,105.800733,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4,Hồ Chí Minh,Bình Tân,Phường Tân Tạo,Đường Lê Đình Cẩn,302/,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,02/07/2025,NaN,NaN,...,12.0,1,Nở hậu,6.0,NaN,Đất ở,chủ đang cho thuê tháng 7 triệu | mặt tiền đườ...,10.759390,106.599232,"[""https://file4.batdongsan.com.vn/crop/600x315..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,Hà Nội,Hà Đông,Phường Quang Trung,Đường Ngô Thì Nhậm,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,1.000000e+10,...,50.0,1,Chữ nhật,5.0,NaN,Đất ở,+ nhà vị trí cực đẹp ngô thì nhậm sát đường qu...,20.968794,105.769142,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4986,Hà Nội,Đống Đa,Phường Ô Chợ Dừa,Phố Xã Đàn,Mặt phố,https://batdongsan.com.vn/ban-nha-rieng-pho-xa...,Rao bán,06/07/2025,NaN,8.700000e+09,...,44.0,1,Chữ nhật,3.0,20.0,Đất ở,- tổng 6 tầng 4 ngủ đầy đủ công năng full nội ...,21.015794,105.832597,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4987,Hồ Chí Minh,Thủ Đức,Phường Trường Thọ,Đường Đặng Văn Bi,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,5.680000e+09,...,NaN,1,Nở hậu,5.0,NaN,Đất ở,"nhà 2 tầng mới keng, xây kiên cố, gồm 3 phòng ...",10.843900,106.762166,"[""https://file4.batdongsan.com.vn/crop/600x315..."
4988,Hồ Chí Minh,Bình Thạnh,Phường 1,Đường Vũ Huy Tấn,Mặt ngõ,https://batdongsan.com.vn/ban-nha-rieng-duong-...,Rao bán,06/07/2025,NaN,NaN,...,17.0,1,Chữ nhật,NaN,NaN,Đất ở,"đang cho thuê 22triệu/tháng, gần chung cư vũ h...",10.796153,106.693391,"[""https://file4.batdongsan.com.vn/crop/600x315..."


In [25]:
clean_file.iloc[1074]['Đường phố']

'Đường Chế Lan Viên'